In [104]:
import pandas as pd
import spacy 
from spacy.lang.es.examples import sentences 
import es_core_news_sm
# Import CountVectorizer and RegexTokenizer
from sklearn.feature_extraction.text import CountVectorizer
import re
# Preprocesado y modelado
# ==============================================================================
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
#nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords 
from nltk import word_tokenize
from string import punctuation


In [105]:
df = pd.read_csv("CSV_Informacion_Categorizada/CR_PE_MX_UY.csv")
df.head(3)

,Unnamed: 0,Id,User,Content,Date,Lang,Sentiment
0,0,768225400254111744,14628107,@NoilyMV yo soy totalmente puntual,Tue Aug 23 23:16:23 +0000 2016,es,NONE
1,1,770077064833671168,713600676,@SandraCauffman Hola Sandrita. No le habia des...,Mon Aug 29 01:54:14 +0000 2016,es,P
2,2,771207534342320128,120940293,Si andan haciendo eso mejor se quedaran callad...,Thu Sep 01 04:46:19 +0000 2016,es,N


In [106]:
stop =stopwords.words('spanish') 

In [107]:
count_vector = CountVectorizer(max_features=1000, lowercase=True, analyzer='word', stop_words= stop,ngram_range=(1,1))

In [117]:
#Limpia la data de signos de puntuacion
doc_complete = df.Content.values
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

doc_clean = [clean(doc).split() for doc in doc_complete]   
df.dropna(inplace=True)
print(doc_clean)

[['noilymv', 'totalmente', 'puntual'], ['sandracauffman', 'hola', 'sandrita', 'habia', 'deseado', 'feliz', 'dia', 'madre', 'tarde', 'seguro¿', 'sabe', 'usted', 'si', 'juno', 'tomado', 'ma', 'fotos'], ['si', 'andan', 'haciendo', 'mejor', 'quedaran', 'calladitas', 'jaja', 'así', 'quemando', 'mismas', 'insinuando', 'unas', 'metidas'], ['pereza', 'quiero', 'choco', 'banano'], ['robertobrenes', 'bueno', 'mayor', 'cuánto', 'campo', 'usted', 'so', 'cartaguito', 'cada', 'aspecto', 'vida', 'visto'], ['pasé', 'día', 'buscando', 'baby', 'lip', 'acabo', 'encontrar', 'mesita', 'noche', 'fijo', 'puse', 'ahí', 'dormida'], ['doriamdiaz', 'halfon', 'germinal', 've', 'mortal', 'mary', 'jo', 'bang', 'volando', 'todavía', 'quedan', 'duluoz', 'salcedo'], ['ahorita', 'van', 'cambiar', 'ligas', 'brakets', 'alfajores', 'argentinos', 'bulto', 'hambre', '¿qué', 'hago', 'puedo', 'esperar'], ['amor', 'paciente', 'bondadoso', 'envidioso', 'orgulloso', 'egoísta', 'guarda', 'rencor', '1', 'corintios', '1345', 'buen'

In [109]:
# ajusta el documento al objeto countVectorizer
count_vector.fit(df['Content'])
names = count_vector.get_feature_names()
names

/home/calybeth/Escritorio/soft/entornos/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['00',
 '10',
 '11',
 '12',
 '14',
 '15',
 '18',
 '20',
 '2016',
 '2017',
 '23',
 '25',
 '26',
 '30',
 'abajo',
 'abrazo',
 'abrazos',
 'aburrida',
 'aburrido',
 'aca',
 'acaba',
 'acabo',
 'acuerdo',
 'acá',
 'adelante',
 'adri_tranqui',
 'agarro',
 'agua',
 'ah',
 'ahi',
 'ahora',
 'ahorita',
 'ahí',
 'aire',
 'ale',
 'alegría',
 'alguien',
 'alguna',
 'algún',
 'allá',
 'alma',
 'alto',
 'am',
 'amiga',
 'amigas',
 'amigo',
 'amigos',
 'amo',
 'amor',
 'anda',
 'andar',
 'ando',
 'ansiedad',
 'aparte',
 'apenas',
 'apoyo',
 'app',
 'aprende',
 'aprender',
 'aqui',
 'aquí',
 'argentina',
 'arriba',
 'asi',
 'así',
 'atrás',
 'aun',
 'aunque',
 'auto',
 'aveces',
 'ay',
 'ayer',
 'ayuda',
 'ayudar',
 'año',
 'años',
 'aún',
 'baja',
 'bajo',
 'banco',
 'bastante',
 'baño',
 'be',
 'bebé',
 'bella',
 'bendiciones',
 'besitos',
 'beso',
 'besos',
 'bien',
 'bonita',
 'bonito',
 'buen',
 'buena',
 'buenas',
 'bueno',
 'buenos',
 'bus',
 'busca',
 'buscando',
 'buscar',
 'bye',
 'cabeza',

In [110]:
#covertir a matriz de enteros
doc_array = count_vector.transform(df['Content']).toarray()
doc_array

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [111]:
# convertir esta tabla en una estructura de datos y nombrar las columnas adecuadamente.
frequency_matrix = pd.DataFrame(data=doc_array, columns=names)
frequency_matrix

,00,10,11,12,14,15,18,20,2016,2017,...,voz,xkidas,xq,youtube,éxito,éxitos,última,último,única,único
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3670,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3671,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3672,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3673,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [112]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['Content'], df['Sentiment'], random_state=1)
print('Número de filas en el conjunto total: {}'.format(df.shape[0]))
print('Número de filas en el conjunto de entrenamiento: {}'.format(X_train.shape[0]))
print('Número de filas en el conjunto de prueba: {}'.format(X_test.shape[0]))

Número de filas en el conjunto total: 3675
Número de filas en el conjunto de entrenamiento: 2756
Número de filas en el conjunto de prueba: 919


In [113]:
# Fit the training data and then return the matrix
training_data = count_vector.fit_transform(X_train)
# Transform testing data and return the matrix. Note we are not fitting the testing data into the CountVectorizer()
testing_data = count_vector.transform(X_test)

In [114]:
#Importaremos el clasificador “MultinomialNB” y ajustaremos los datos de entrenamiento
#en el clasificador usando fit().
from sklearn.naive_bayes import MultinomialNB
naive_bayes = MultinomialNB()
naive_bayes.fit(training_data,y_train)

MultinomialNB()

In [115]:
#redicciones en los datos de prueba almacenados en ‘testing_data’ usando predict().
predictions = naive_bayes.predict(testing_data)

In [116]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('Accuracy score: ', format(accuracy_score(y_test, predictions)))
print('Precision score: ', format(precision_score(y_test, predictions, average="macro")))
print('Recall score: ', format(recall_score(y_test, predictions,average="weighted")))
print('F1 score: ', format(f1_score(y_test, predictions,average="micro")))

Accuracy score:  0.5233949945593036
Precision score:  0.43418928134302404
Recall score:  0.5233949945593036
F1 score:  0.5233949945593036
